In [1]:
import pandas as pd
import numpy as np
from scipy import spatial

number_of_data_sets = 10
data_sets_dir = 'data/sets/'
k = 5

i = 0
test_file = data_sets_dir + str(i) + '/test.csv'
train_file = data_sets_dir + str(i) + '/train.csv'
test_df = pd.read_csv(test_file)
train_df = pd.read_csv(train_file)

In [4]:
target_userid = 'A3Q4S5DFVPB70D'
common_products = train_df[train_df.review_userid == target_userid].product_productid
userid_array = train_df[train_df.product_productid.isin(common_products)].review_userid.unique()

In [5]:
def calc_sim(s1, s2):
    def get_norm(s):
        return (s - s.mean()).fillna(0)

    norm_s1 = get_norm(s1)
    norm_s2 = get_norm(s2)

    if norm_s1.dot(norm_s2) == 0:
        result = 0
    else:
        result = 1 - spatial.distance.cosine(norm_s1, norm_s2)
    return result

In [6]:
result_sim = pd.Series()
for index, userid in enumerate(userid_array):
    df = train_df[train_df.review_userid.isin([target_userid, userid])]
    # convert to pivot table to simplify calculation
    table = pd.pivot_table(
        df,
        values='review_score',
        index=['product_productid'],
        columns=['review_userid']
    )

    # calculate the similarity and store the result
    similarity = calc_sim(table[target_userid], table[userid])
    result_sim.set_value(userid, similarity)
result_sim = result_sim.drop(target_userid)

In [8]:
productid = 'B00006HAXW'
rated_users = train_df[train_df.product_productid == productid].review_userid.unique()
knn = result_sim.get(rated_users).sort_values(ascending=False)[:k].fillna(0)

In [14]:
userid_array

array(['AD4CDZK7D31XP', 'A3Q4S5DFVPB70D', 'A2P7UB02HAVEPB',
       'A2TX99AZKDK0V7', 'A1FRPGQYQTAOR1', 'A1RSDE90N6RSZF',
       'A1OUBOGB5970AO', 'A3NPHQVIY59Y0Y', 'AFKMBAY28XO8A',
       'A66KMXH9V7OGU', 'AFJ27ZV9183B8', 'AXMKAXC0TR9AW'], dtype=object)

In [19]:
train_df[train_df.review_userid == target_userid]

,product_productid,review_userid,review_profilename,review_helpfulness,review_score
6757142,B00006HB30,A3Q4S5DFVPB70D,Joseph P. Aiello,26/26,5


In [20]:
test_df[test_df.review_userid == target_userid]

,product_productid,review_userid,review_profilename,review_helpfulness,review_score
0,B00006HAXW,A3Q4S5DFVPB70D,Joseph P. Aiello,26/26,5
